In [9]:
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
import cv2
from skimage.filters import sobel
from skimage.feature import graycomatrix, graycoprops
import os

# Proceso de Entrenamiento
## Parte 1: Agregar las imágenes a un arreglo
### En cuadernos de notas anteriores se realizó el proceso de preprocesamiento de datos necesario para poder aislar lo mayor posible al cerebro del resto del tejido circundante. Con las imágenes ya preprocesadas, estas pasan a la siguiente parte del pipeline. Estas imágenes serán convertidas a formato de openCV para que puedan ser leídas con facilidad. Este arreglo será transformado a un dataframe en los últimos pasos del pipeline.

In [10]:
train_images = [] #Inicializar arreglos en cero
train_labels = [] 

test_images = []
test_labels = []

from pathlib import Path

path_train = Path("datasets/brain-tumor-mri-datasets/Training")
path_test = Path("datasets/brain-tumor-mri-datasets/Testing")

#Labels de entrenamiento
for label in os.listdir(path_train):
    for img in os.listdir(os.path.join(path_train,label)):
        image = cv2.imread(os.path.join(path_train,label,img))
        train_images.append(image)
        train_labels.append(label)


#Hacer lo mismo con los de prueba
for label in os.listdir(path_test):
    for img in os.listdir(os.path.join(path_test,label)):
        image = cv2.imread(os.path.join(path_test,label,img))
        test_images.append(image)
        test_labels.append(label)

train_images = np.array(train_images)
train_labels = np.array(train_labels)
test_images = np.array(test_images)
test_labels = np.array(test_labels)
                           
print("Imprimir la forma de cada arreglo")
print(f"train_images: {train_images.shape} train_labels: {train_labels.shape} test_images: {test_images.shape} test_labels: {test_labels.shape}")


Imprimir la forma de cada arreglo
train_images: (5465, 256, 256, 3) train_labels: (5465,) test_images: (1253, 256, 256, 3) test_labels: (1253,)


In [16]:
#Transformar los labels de forma codificada
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
le.fit(train_labels)
train_labels_encoded = le.transform(train_labels)
le.fit(test_labels)
test_labels_encoded = le.transform(test_labels)

array([0, 1, 2, 3])

# Parte 2: Extracción de Características
## Se emplea en conjunto pandas y las librerías de skimage (usada para extraer características y manipulación de  imágenes) con el fin de extraer las características más fundamentales de las imágenes. Para esto usaré la técnica de la Gray Level Co-occurence Matrix, una de las técnica que expliqué en mis primeras presentaciones. 